# Spotify Data Extraction

In [158]:
import spotipy
import pandas as pd
import math
from spotipy.oauth2 import SpotifyClientCredentials

import warnings
warnings.filterwarnings("ignore")

pd.options.display.max_rows = 999
pd.options.display.max_columns = 999

In [35]:
sp = spotipy.Spotify(
    client_credentials_manager=SpotifyClientCredentials(
        client_id='c3c6cbaa304c4c86b120e8d9603d44a4', 
        client_secret='b8819ea70b3142819602499ee139c24f'))

nirvana_uri = 'spotify:artist:gB8MJK4WTc2PGssawZYu_w'

In [36]:
alice_uri = '64tNsm6TnZe2zpcMVMOoHL'

all_tracks = []

def get_data(results):
    albums = results['items']
    
    while results['next']:
        results = sp.next(results)
        albums.extend(results['items'])
    
    for album in albums:
        result = sp.album_tracks(album['uri'])
        tracks = result['items']
        
        while result['next']:
            result = sp.next(result)
            tracks.extend(result['items'])
            
        for track in tracks:
            new_track = {}
            new_track['track_name'] = track['name']
            new_track['album_name'] = album['name']
            new_track['album_type'] = album['album_type']
            new_track['release_year'] = album['release_date'][0:4]
            new_track['artists'] = ', '.join([artist['name'] for artist in track['artists']])
            new_track['duration'] = math.floor(track['duration_ms']/1000)
    
            audio = sp.audio_features(tracks=track['id'])
            new_track['acousticness'] = audio[0]['acousticness']
            new_track['danceability'] = audio[0]['danceability']
            new_track['energy'] = audio[0]['energy']
            new_track['instrumentalness'] = audio[0]['instrumentalness']
            new_track['key'] = audio[0]['key']
            new_track['liveness'] = audio[0]['liveness']
            new_track['loudness'] = audio[0]['loudness']
            new_track['mode'] = audio[0]['mode']
            new_track['speechiness'] = audio[0]['speechiness']
            new_track['tempo'] = audio[0]['tempo']
            new_track['time_signature'] = audio[0]['time_signature']
            new_track['valence'] = audio[0]['valence']
            
            all_tracks.append(new_track)

r = sp.artist_albums(alice_uri, album_type='album')
get_data(r)
r = sp.artist_albums(alice_uri, album_type='compilation')
get_data(r)

In [37]:
df = pd.DataFrame(all_tracks)

### Cleaning the dataset

In [38]:
# Dropping Discover Beyond, Discover More and Discover Further albums
df.drop(df.loc[df['album_name'].isin(['Discover Beyond','Discover More','Discover Further'])].index, inplace=True)
df.reset_index(drop=True, inplace=True)

# Renaming album types
df.loc[0:34,'album_type'] = 'studio album'
df.loc[35:47, 'album_type'] = 'live album'
df.loc[48:59, 'album_type'] = 'studio album'
df.loc[60:66, 'album_type'] = 'extended play'
df.loc[67:91, 'album_type'] = 'studio album'
df.loc[92:149, 'album_type'] = 'compilation album'
df.loc[150:163, 'album_type'] = 'live album'
df.loc[164:191, 'album_type'] = 'compilation album'
df.loc[192:, 'album_type'] = 'box set'
df.loc[67:79, 'album_name'] = df.loc[67:79, 'album_name'].str.split(' ').str[0]

# Cleaning track names
df.loc[35:47,'track_name'] = df.loc[35:47,'track_name'].str.split(' - ').str[0]
df.loc[67:79, 'track_name'] = df.loc[67:79, 'track_name'].str.split(' \(2022 Remaster\)').str[0]
df.loc[150:163, 'track_name'] = df.loc[150:163, 'track_name'].str.split(' - ').str[0]

# Cleaning data for merging purpose
df.loc[df['track_name'].str.contains('Demo'), 'track_name'] = df.loc[df['track_name'].str.contains('Demo'),'track_name'].str.split(' - ').str[0]
df.loc[106, 'track_name'] = 'Queen of the Rodeo'

df.loc[df['track_name'].str.contains('Unplugged'), 'track_name'] = df.loc[df['track_name'].str.contains('Unplugged'), 'track_name'].str.split(' - ').str[0]
df.loc[df['album_name'].str.contains('Box'), 'album_name'] = df.loc[df['album_name'].str.contains('Box'), 'album_name'].str.split(' - ').str[0]

df.loc[df['album_name'] == 'Live', 'release_year'] = str(2000) # 1993 was incorrect
df.loc[df['album_name'].str.contains('Esse'), 'release_year'] = str(2006) # 1990 was incorrect
df.loc[df['album_name'].str.contains('Safe'), 'release_year'] = str(1999) # 1990 was incorrect

df.loc[[199, 200], 'track_name'] = df.loc[[199, 200], 'track_name'].str.split(' - ').str[0]

In [39]:
df.to_csv('alice_spotify.csv', index=False)
spot = df

# Wikipedia Data Extraction

In [40]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import re

In [41]:
url = 'https://en.wikipedia.org/wiki/List_of_songs_recorded_by_Alice_in_Chains'
response = requests.get(url)
soup = BeautifulSoup(response.content, 'html.parser')

In [42]:
table = soup.find_all('table', class_ = 'wikitable')[0]

rows = table.find_all('tr')

In [43]:
# This code was used from this StackOverflow thread
# https://stackoverflow.com/questions/9978445/parsing-a-table-with-rowspan-and-colspan
# Answer by: Joshua

def table_to_2d(table_tag):
    rows = table_tag("tr")
    cols = rows[0](["td", "th"])
    table = [[None] * len(cols) for _ in range(len(rows))]
    for row_i, row in enumerate(rows):
        for col_i, col in enumerate(row(["td", "th"])):
            insert(table, row_i, col_i, col)
    return table


def insert(table, row, col, element):
    if row >= len(table) or col >= len(table[row]):
        return
    if table[row][col] is None:
        value = element.get_text()
        table[row][col] = value
        if element.has_attr("colspan"):
            span = int(element["colspan"])
            for i in range(1, span):
                table[row][col+i] = value
        if element.has_attr("rowspan"):
            span = int(element["rowspan"])
            for i in range(1, span):
                table[row+i][col] = value
    else:
        insert(table, row, col + 1, element)

In [44]:
t = table_to_2d(table)

In [45]:
df = pd.DataFrame(t)

In [46]:
column_names = df.iloc[0,:].str.strip().str.rstrip('\(s\)').str.lower()
df.columns = column_names
df.drop(0, axis=0, inplace=True)

In [47]:
df = df.apply(lambda x: x.str.strip())

In [48]:
# Transforming author column
def transform_author(x):
    x = x.split('/')
    x = [y.strip() for y in x]
    z = []
    if 'Staley' in x:
        z.append('Layne Staley')
    if 'Cantrell' in x:
        z.append('Jerry Cantrell')
    if 'Inez' in x:
        z.append('Mike Inez')
    if 'Kinney' in x:
        z.append('Sean Kinney')
    if 'DuVall' in x:
        z.append('William DuVall')
    if 'Jett Silver' in x:
        z.append('Jett Silver')
    return ', '.join(z)

df['author'] = df['author'].apply(transform_author)

In [49]:
# Creating album_name column from release column
df['album_name'] = df['release'].str.split('\(').str[0]

In [50]:
# Creating release_year column from release column
df['release_year'] = df['release'].str.split('\(').str[1].str.split(', ').str[1].str[0:4]

In [51]:
# Dropping release column
df.drop('release', axis=1, inplace=True)

In [52]:
# Dropping recorded and location columns
df.drop(['recorded','location'], axis=1, inplace=True)

In [53]:
# Dropping time column
df.drop('time', axis=1, inplace=True)

In [54]:
# Transforming producer column
df['producer'].value_counts()

def transform_producer(x):
    x = x.split('/')
    x = [y.strip() for y in x]
    for i in range(len(x)):
        if x[i] == 'Raskulinecz': x[i] = 'Nick Raskulinecz'
        if x[i] == 'Wright': x[i] = 'Toby Wright'
        if x[i] == 'Jerden': x[i] = 'Dave Jerden'
        if x[i] == 'Coletti': x[i] = 'Alex Coletti'
        if x[i] == 'Wilson': x[i] = 'Tony Wilson'
        if x[i] == 'Naficy': x[i] = 'Mark Naficy'
    return ', '.join(x)

df['producer'] = df['producer'].apply(transform_producer)

In [55]:
# Cleaning album_name column
df.loc[df['album_name'].str.contains('Nothing Safe'), 'album_name'] = 'Nothing Safe'
df.loc[df['album_name'].str.contains('Unplugged'), 'album_name'] = 'Unplugged'

df['album_name'] = df['album_name'].apply(str.strip)

In [56]:
df.to_csv('alice_wiki.csv', index=False)
wiki = df

In [57]:
# Cleaning data for merging purpose

wiki.loc[100,'title'] = 'Hung on a Hook'
wiki.loc[29, 'title'] = 'Black Gives Way to Blue - Piano Mix'
wiki.loc[29, 'album_name'] = 'Black Gives Way to Blue'
wiki.loc[wiki['album_name'].str.contains('Jar'),'album_name'] = 'Jar Of Flies'
wiki.loc[wiki['album_name'] == 'Alice in Chains','album_name'] = 'Alice In Chains'
wiki.loc[[30, 56, 60, 88, 186, 187, 89, 189, 200],'title'] = wiki.loc[[30, 56, 60, 88, 186, 187, 89, 189, 200],'title'].str.title()
wiki.loc[[67,77, 73, 70],'album_name'] = 'Music Bank'
wiki.loc[[68, 78, 74],'album_name'] = 'The Essential Alice In Chains'
wiki.loc[30, 'title'] = 'Bleed the Freak'
wiki.loc[[219, 220, 221], 'title'] = 'What the Hell Have I - Remix'
wiki.loc[[1,2, 3], 'title'] = 'A Little Bitter - Remix'
wiki.loc[13, 'title'] = 'Again - Tattoo of Pain Mix'

wiki.loc[wiki['album_name'].str.contains('Esse'), 'album_name'] = wiki.loc[wiki['album_name'].str.contains('Esse'), 'album_name'].str.title()

wiki.loc[53, 'title'] = wiki.loc[53, 'title'].split(' ')[0]

wiki.loc[wiki['title'].str.contains('Have')]

,title,author,producer,album_name,release_year
101,I Can't Have You Blues,Jerry Cantrell,"Alice in Chains, Parashar",Music Bank,1999
218,What The Hell Have I,Jerry Cantrell,Alice in Chains,Last Action Hero: Music from the Original Moti...,1993
219,What the Hell Have I - Remix,Jerry Cantrell,Alice in Chains,Nothing Safe,1999
220,What the Hell Have I - Remix,Jerry Cantrell,Alice in Chains,Music Bank,1999
221,What the Hell Have I - Remix,Jerry Cantrell,Alice in Chains,The Essential Alice In Chains,2006


In [58]:
alice_v1 = spot.merge(wiki, how='left', left_on=['track_name', 'album_name', 'release_year'], right_on=['title', 'album_name', 'release_year']).drop('title', axis=1)
alice_v1.to_csv('alice_v1.csv', index=False)

# AllMusic Data Extraction

In [59]:
from requests_html import HTMLSession
from bs4 import BeautifulSoup
import pandas as pd

In [60]:
session = HTMLSession()
url = 'https://www.allmusic.com/artist/alice-in-chains-mn0000007920/discography/all'
response = session.get(url)
soup = BeautifulSoup(response.text, 'html.parser')

In [61]:
def get_page_data(url):
    """
    Input a page URL to get a BeautifulSoup object from that page.
    """
    response = session.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    return soup

In [62]:
def get_album_page_links(soup):
    """
    Input a BeautifulSoup object to get links to all albums present on that discography page
    """
    links = []
    data = soup.find_all('td', class_ = 'title')
    for album in data:
        if album.find('a'):
            links.append(album.find('a')['href'])
    return links

def get_track_page_links(soup):
    """
    Input a BeautifulSoup object to get links to all tracks present on that album page
    """
    links = []
    data = soup.find_all('div', class_ = 'title')
    for track in data:
        if track.find('a'):
            links.append(track.find('a')['href'])
    return links

album_links = get_album_page_links(soup)

In [63]:
def calculate_rating(classes):
    """
    A function to parse classes from AllMusic website as a rating on a scale from [0.5-5.0].
    This function will expect a list of css classes as a parameter, and output 
    rating according to present class.
    """
    if 'rating-unrated' in classes: rating = ''
    elif 'rating-allmusic-0' in classes: rating = ''
    elif 'rating-allmusic-1' in classes: rating = 1.0
    elif 'rating-allmusic-2' in classes: rating = 1.5
    elif 'rating-allmusic-3' in classes: rating = 2.0
    elif 'rating-allmusic-4' in classes: rating = 2.5
    elif 'rating-allmusic-5' in classes: rating = 3.0
    elif 'rating-allmusic-6' in classes: rating = 3.5
    elif 'rating-allmusic-7' in classes: rating = 4.0
    elif 'rating-allmusic-8' in classes: rating = 4.5
    elif 'rating-allmusic-9' in classes: rating = 5.0
    return rating

In [64]:
def get_track_data(url):
    """
    This function expects a track URL as a parameter. It returns information about the track
    as a dictionary"""
    overview_page = get_page_data(url)
    attributes_page = get_page_data(str(url +  '/attributes'))
        
    track_title = overview_page.find('h1').text.strip()
    track_composers = [x.a.text for x in overview_page.find_all('p', class_ = 'song-composer')]
    genres = [x.text[:-4] for x in attributes_page.find('div', class_ = 'attribute-tab-genres').find_all('a')]
    styles = [x.text[:-4] for x in attributes_page.find('div', class_ = 'attribute-tab-styles').find_all('a')] 
    moods = [x.text[:-4] for x in attributes_page.find('div', class_ = 'attribute-tab-moods').find_all('a')] 
    themes = [x.text[:-4] for x in attributes_page.find('div', class_ = 'attribute-tab-themes').find_all('a')] 
        
    return {
        'track_title' : track_title,
        'track_composers' : track_composers,
        'track_genres':genres, 
        'track_styles':styles, 
        'track_moods':moods, 
        'track_themes':themes
    }
        

In [65]:
def get_data(url):
    """
    This function expects an album page URL. It will scrape all data about the album and all tracks from that album.
    It will return a list of all tracks from that album with its corresponding information as a dictionary"""

    # ALBUM data
    album_page = get_page_data(url)
    album_title = album_page.find('h1', class_ = 'album-title').text.strip()
    album_release_date = [album_page.find('div', class_ = 'release-date').span.text if album_page.find('div', class_ = 'release-date') else '']
    album_duration = [album_page.find('div', class_ = 'duration').text.strip() if album_page.find('div', class_ = 'duration') else None ]
    album_genre = [album_page.find('div', class_ = 'genre').div.a.text if album_page.find('div', class_ = 'genre') else '']
    album_styles = [[a.text for a in album_page.find('div', class_ = 'styles').find_all('a')] if album_page.find('div', class_ = 'styles') else '']
    album_recording_date = [album_page.find('div', class_ = 'recording-date').div.text if album_page.find('div', class_ = 'recording-date') else '']
    album_recording_location = [album_page.find('div', class_ = 'recording-location').li.text.strip() if album_page.find('div', class_ = 'recording-location') else '']
    album_moods = [x.text.strip() for x in album_page.find_all('span', class_ = 'mood')]
    album_themes = [x.text.strip() for x in album_page.find_all('span', class_ = 'theme')]
    album_rating = calculate_rating(album_page.find('div', class_ = 'allmusic-rating').get('class'))

    # TRACK data
    all_tracks_data = []
    track_links = get_track_page_links(album_page)
    if len(track_links) > 0:
        for track_link in track_links:
            all_tracks_data.append(get_track_data(track_link))

    # FINAL data
    final_data = []
    
    for track in all_tracks_data:
        final_data.append({
            'track_title' : track['track_title'],
            'track_composers' : track['track_composers'],
            'track_genres': track['track_genres'], 
            'track_styles': track['track_styles'], 
            'track_moods': track['track_moods'], 
            'track_themes': track['track_themes'],
            'album_title': album_title,
            'album_release_date': album_release_date,
            'album_duration' : album_duration,
            'album_genres' : album_genre,
            'album_styles' : album_styles,
            'album_recording_date' : album_recording_date,
            'album_recording_location' : album_recording_location,
            'album_moods' : album_moods,
            'album_themes' : album_themes,
            'album_rating' : album_rating
        })
    return final_data

### Fetching all data from AllMusic

In [66]:
all_tracks = []
for i in range(len(album_links)):
    all_tracks.extend(get_data(album_links[i]))

### Cleaning fetched data

In [141]:
df = pd.DataFrame(all_tracks)

In [142]:
# df.drop('album_release_date', axis=1, inplace=True)
df.drop('album_genres', axis=1, inplace=True)

In [143]:
print(f'Columns:\t{df.shape[1]}\nRows:\t\t{df.shape[0]}')

Columns:	15
Rows:		472


In [144]:
df['track_composers'] = df['track_composers'].apply(lambda x: ', '.join(x))
df['track_genres'] = df['track_genres'].apply(lambda x: ', '.join(x))
df['track_styles'] = df['track_styles'].apply(lambda x: ', '.join(x))
df['track_moods'] = df['track_moods'].apply(lambda x: ', '.join(x))
df['track_themes'] = df['track_themes'].apply(lambda x: ', '.join(x))
df['album_duration'] = df['album_duration'].apply(lambda x: str(x[0])[9:])
df['album_styles'] = df['album_styles'].apply(lambda x: ''.join(x[0]))
df['album_recording_date'] = df['album_recording_date'].apply(lambda x: ', '.join(x))
df['album_release_date'] = df['album_release_date'].apply(lambda x: str(x)[-6:-2])
df['album_recording_location'] = df['album_recording_location'].apply(lambda x: ', '.join(x))
df['album_moods'] = df['album_moods'].apply(lambda x: ', '.join(x))
df['album_themes'] = df['album_themes'].apply(lambda x: ', '.join(x))

In [145]:
# Cleaning the missing data format
df.loc[df['track_genres'] == 'Would you like to contrib', 'track_genres'] = ''
df.loc[df['track_styles'] == 'Would you like to contrib', 'track_styles'] = ''
df.loc[df['track_moods'] == 'Would you like to contrib', 'track_moods'] = ''
df.loc[df['track_themes'] == 'Would you like to contrib', 'track_themes'] = ''

In [151]:
# Cleaning album names

df.loc[df['album_title'] == 'MTV Unplugged', 'album_title'] = 'Unplugged'
df.loc[df['album_title'] == 'Alice in Chains', 'album_title'] = 'Alice In Chains'
df.loc[df['album_title'] == 'Jar of Flies', 'album_title'] = 'Jar Of Flies'
df.loc[df['album_title'] == 'The Essential Alice in Chains', 'album_title'] = 'The Essential Alice In Chains'

In [161]:
# Cleaning missmatched data

df.loc[df['album_title'] == 'The Essential Alice In Chains', 'album_release_date' ] = '2006'

In [259]:
# Cleaning track names

df.loc[df['track_title'] == 'Rainer Fog', 'track_title'] = 'Rainier Fog'
df.loc[387,'track_title'] = 'Breath on a Window'
df.loc[390,'track_title'] = 'Hung on a Hook'
df.loc[[89, 150], 'track_title'] = 'The Killer Is Me'
df.loc[27, 'track_title'] = 'Untitled'

list = [2, 3, 31, 38, 47, 59, 66, 81, 114, 123, 140, 262, 274]
df.loc[list, 'track_title'] = df.loc[list, 'track_title'].str.title()

In [261]:
df.to_csv('alice_allmusic.csv', index=False)
allmusic = df

In [262]:
allmusic.drop_duplicates(inplace=True)

In [264]:
alice_v2 = alice_v1.merge(allmusic, how='left', left_on=['track_name', 'album_name', 'release_year'], right_on=['track_title', 'album_title', 'album_release_date']).drop(['track_title','album_title', 'album_release_date'], axis=1)
alice_v2.to_csv('alice_v2.csv', index=False)

# Billboard data extraction

In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
import pandas as pd

In [16]:
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))

driver.get("https://www.billboard.com/artist/alice-in-chains/")

In [3]:
def extract_data(chart_name = ""):
    data = []
    
    table = driver.find_element(By.CLASS_NAME, 'artist-chart-history-items')
    rows = table.find_elements(By.XPATH, '*')
    
    for row in rows:
        track = {}
        track_name = row.find_element(By.ID, 'title-of-a-story')
        debut_date = row.find_element(By.CLASS_NAME, 'artist-chart-row-debut-date')
        peak_pos = row.find_element(By.CLASS_NAME, 'artist-chart-row-peak-pos')
        peak_date = row.find_element(By.CLASS_NAME, 'artist-chart-row-peak-date')
        weaks_on_chart = row.find_element(By.CLASS_NAME, 'artist-chart-row-week-on-chart')

        track['chart'] = chart_name
        track['track_name'] = track_name.text
        track['debut_date']  = debut_date.text
        track['peak_pos'] = peak_pos.text
        track['peak_date'] = peak_date.text
        track['weaks_on_chart'] = weaks_on_chart.text
        data.append(track)

    
    return data
        
        # print(row)
        
    print()

In [25]:
filter = driver.find_element(By.XPATH, '//*[@id="artist-chart-selector"]')
current_li = 1

all_data = []

while(True):
    if current_li == 55:
        break
        
    filter.click()
    
    li_path = f'//*[@id="main-wrapper"]/main/div[2]/div[2]/div/div[2]/div[1]/nav/ul/li[{current_li}]/a'
    new_li = driver.find_element(By.XPATH, li_path)
    new_li.click()

    print(f"Chart: {new_li.get_attribute('text').strip()}", end='\r')
    
    all_data.extend(extract_data(chart_name=new_li.get_attribute('text').strip()))
    
    current_li += 1
driver.refresh()

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="main-wrapper"]/main/div[2]/div[2]/div/div[2]/div[1]/nav/ul/li[41]/a"}
  (Session info: chrome=117.0.5938.152); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00FDCFE3+45267]
	(No symbol) [0x00F69741]
	(No symbol) [0x00E5BE1D]
	(No symbol) [0x00E8ED30]
	(No symbol) [0x00E8F1FB]
	(No symbol) [0x00EBF772]
	(No symbol) [0x00EAB084]
	(No symbol) [0x00EBDDDA]
	(No symbol) [0x00EAAE36]
	(No symbol) [0x00E8674E]
	(No symbol) [0x00E878ED]
	GetHandleVerifier [0x01295659+2897737]
	GetHandleVerifier [0x012DE78B+3197051]
	GetHandleVerifier [0x012D8571+3171937]
	GetHandleVerifier [0x01065E40+606000]
	(No symbol) [0x00F7338C]
	(No symbol) [0x00F6F508]
	(No symbol) [0x00F6F62F]
	(No symbol) [0x00F61D27]
	BaseThreadInitThunk [0x7600FCC9+25]
	RtlGetAppContainer

In [69]:
import pandas as pd

df = pd.DataFrame(all_data)

In [70]:
df

,chart,track_name,debut_date,peak_pos,peak_date,weaks_on_chart
0,Billboard Hot 100,Check My Brain,10.03.09,92,10.17.09,4
1,Billboard 200,Dirt,10.17.92,6,10.17.92,104
2,Billboard 200,Facelift,04.27.91,42,07.06.91,60
3,Billboard 200,Jar Of Flies (EP),02.12.94,1,02.12.94,59
4,Billboard 200,Alice In Chains,11.25.95,1,11.25.95,46
...,...,...,...,...,...,...
115,LyricFind U.S.,Nutshell,10.23.21,15,10.23.21,1
116,Greatest of All Time Mainstream Rock Artists,Alice In Chains,05.29.21,23,05.29.21,1
117,Germany Albums,Rainier Fog,09.08.18,8,09.08.18,1
118,The Official U.K. Albums Chart,Rainier Fog,09.08.18,9,09.08.18,1


In [71]:
df['chart'].value_counts()
useful_charts = [
    'Mainstream Rock Airplay',
    'Alternative Airplay',
    'Radio Songs', 
    'LyricFind U.S.',
    'Billboard Hot 100',
    'Rock Digital Song Sales',
    'Hot Alternative Songs',
    'LyricFind Global',
    'Alternative Digital Song Sales',
    'Billboard Global 200',
    'Canadian Digital Song Sales',
    'Hot Rock Songs',
    'Hot Rock & Alternative Songs',
    'Digital Song Sales',
    'Rock Streaming Songs',
    'Alternative Streaming Songs'
    ]
indices = df.loc[df['chart'].isin(useful_charts),].index

In [72]:
df = df.loc[indices,]
df.reset_index(drop=True, inplace=True)

In [73]:
df.to_csv('billboard.csv',index=False)

### Creating Billboard Charts columns

In [75]:
df['chart'] = df['chart'].apply(lambda x: x.replace('\xa0',' '))

In [150]:
alice = pd.read_csv('alice_v2.csv')
alice.drop_duplicates(subset=['track_name', 'album_name', 'release_year'], inplace=True)

In [166]:
for i in range(df.shape[0]):

    if alice.loc[alice['track_name'].str.title() == df.loc[i,'track_name'],].index.size == 1:
        # If song has only one occurence in alice_v2 then input Billboard data there
        chart = df.loc[i,'chart'].lower().replace(' ','_')
        track = df.loc[i,'track_name']
        index = alice.loc[alice['track_name'].str.title() == df.loc[i,'track_name'],].index[0]

        if chart + '_debut_date' not in alice.columns: alice[chart + '_debut_date'] = ''
        alice[chart + '_debut_date'][index] = df.loc[i,'debut_date']

        if chart + '_peak_pos' not in alice.columns: alice[chart + '_peak_pos'] = ''
        alice[chart + '_peak_pos'][index] = df.loc[i,'peak_pos']

        if chart + '_peak_date' not in alice.columns: alice[chart + '_peak_date'] = ''
        alice[chart + '_peak_date'][index] = df.loc[i,'peak_date']

        if chart + '_weaks_on_chart' not in alice.columns: alice[chart + '_weaks_on_chart'] = ''
        alice[chart + '_weaks_on_chart'][index] = df.loc[i,'weaks_on_chart']
    else:
        # If song has multiple occurences in alice_v2 find the original (lowest release year)
        chart = df.loc[i,'chart'].lower().replace(' ','_')
        track = df.loc[i,'track_name']
        original_release_year = min(alice.loc[alice.loc[alice['track_name'].str.title() == track,].index, 'release_year'])
        index = alice.loc[(alice['release_year'] == original_release_year) & (alice['track_name'] == track),].iloc[:,:5].index

        if chart + '_debut_date' not in alice.columns: alice[chart + '_debut_date'] = ''
        alice[chart + '_debut_date'][index] = df.loc[i,'debut_date']

        if chart + '_peak_pos' not in alice.columns: alice[chart + '_peak_pos'] = ''
        alice[chart + '_peak_pos'][index] = df.loc[i,'peak_pos']

        if chart + '_peak_date' not in alice.columns: alice[chart + '_peak_date'] = ''
        alice[chart + '_peak_date'][index] = df.loc[i,'peak_date']

        if chart + '_weaks_on_chart' not in alice.columns: alice[chart + '_weaks_on_chart'] = ''
        alice[chart + '_weaks_on_chart'][index] = df.loc[i,'weaks_on_chart']
        
        
        
    # print(alice.loc[alice['track_name'].str.title() == df.loc[i,'track_name'],].index.size)


In [168]:
alice.to_csv('alice_v3.csv', index=False)